In [2]:
import os
import csv

In [5]:
current_database_folder = "C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24"

In [7]:
print(current_database_folder)

C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24


In [35]:
for root, dirs, files in os.walk(current_database_folder):
    for dir_name in dirs:
        # Extract part before underscore
        dvd_folder_name = dir_name.split('_')[0]
        if dvd_folder_name.startswith('EastDVD') or dvd_folder_name.startswith('WestDVD'):
            dvd_path = os.path.join(root, dir_name)
            for root, dirs, files in os.walk(dvd_path):
                for dir_name in dirs:
                    dir_path = os.path.join(root, dir_name)
                    print(dir_path)
                break
                #for file in files:

C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\RM-ARC
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\RM-ATL
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\RM-CEN
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\V-ATL-A
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\V-ATL-B
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\V-ATL-C
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\V-CEN-A
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\V-CEN-B
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\V-CEN-C
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409\V-NOR-A
C:/Users/wayne/OneDrive/Desktop/CHS DVD Reader/East_West_DVD_Apr24\EastDVD_20240409